In [1]:
# Imports
import open3d as o3d
import numpy as np
import helper_functions
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Getting the whole cave
original_point_cloud = o3d.io.read_point_cloud('01_VersChezLeBrandt_full.ply')
# Draw
#o3d.visualization.draw_geometries([pcd])
#stripped_cave = helper_functions.get_cave_section(original_point_cloud,0,1)
#o3d.visualization.draw_geometries([original_point_cloud])

In [16]:
# Downsampling point cloud
downsampled_point_cloud = o3d.geometry.PointCloud()
downsampled_point_cloud.points = original_point_cloud.points
downsampled_point_cloud = downsampled_point_cloud.voxel_down_sample(0.1)

In [17]:
#o3d.visualization.draw_geometries([downsampled_point_cloud])

In [4]:
# Selecting 23% of the cave along its longest axis, the section selected starts at 67% of the caves longest axis
cave_section_optimized = helper_functions.crop_main_cloud(original_point_cloud,downsampled_point_cloud,0.23,0,-0.17)
# Obtaining bounding box for the cave section
starting_bounding_box = cave_section_optimized.get_minimal_oriented_bounding_box()
starting_bounding_box.color = [0, 1, 0]
# Draw
#o3d.visualization.draw_geometries([cave_section, starting_bounding_box])

In [5]:
# Creating and cleaning the mesh that results from our cave section
original_mesh = helper_functions.get_poisson_mesh_from_point_cloud(cave_section_optimized)
clean_mesh = helper_functions.clean_mesh(original_mesh)
# For visualizing parts that we removed
removed_mesh_parts = helper_functions.clean_mesh(original_mesh, False)
removed_mesh_parts.paint_uniform_color([1,0, 0])

#o3d.visualization.draw_geometries([original_mesh])
#o3d.visualization.draw_geometries([clean_mesh])
#o3d.visualization.draw_geometries([clean_mesh, removed_mesh_parts])

TriangleMesh with 15757 points and 858 triangles.

In [6]:
# Obtaining cave section surface area
surface_area = helper_functions.compute_surface_area(clean_mesh)

In [7]:
surface_area

1317.5273248466435

In [8]:
# Obtaining volume
volume, boxes, meshes, hulls = helper_functions.get_volume_of_cave_section(starting_bounding_box, clean_mesh, 0.01, 2, 0)

In [9]:
volume

1666.015297681442

In [86]:
# For images
#translated_hulls = copy.deepcopy(hulls)
#starting_bounding_box.color = [1, 0, 0]
#o3d.visualization.draw_geometries([clean_mesh] + boxes[70:80] + hulls[70:80] + [starting_bounding_box] + boxes[70:80] + hulls[70:80] +[hull.translate([0,0,-15]) for hull in translated_hulls[70:80]])

In [10]:
# Obtaining caves skeleton
line_skeleton, mesh_skeleton = helper_functions.create_cave_skeleton(cave_section_optimized, clean_mesh, longest_axis_index = 0)
o3d.visualization.draw_geometries([line_skeleton, mesh_skeleton])

In [11]:
# Obtaining cave lenght
cave_lenght = helper_functions.compute_total_line_length(line_skeleton)

In [12]:
cave_lenght

46.72862365705156

In [13]:
# Obtaining roundnes
r1, r2 = helper_functions.obtain_roundness(helper_functions.group_points_by_x_coordinate(original_mesh))
roundnes = np.mean(r1+r2)
roundnes

0.6002850017250969